## Preprocess

In [42]:
import pickle, random, re
pfile = open('saved/lyrics_filtered.pkl', 'rb')

lyrics = pickle.load(pfile)
random.shuffle(lyrics)

dev_lyrics = lyrics[:500]
test_lyrics = lyrics[:5000]

In [2]:
import numpy as np

from gensim.models import Word2Vec

import thulac

In [6]:
class Doc():
    
    model = Word2Vec.load('saved/word2vec_model')
    cut = thulac.thulac(seg_only=True)  #只进行分词，不进行词性标注\n"
    
    corpus = []
    
    def __init__(self, document, tokenizer = 'char'):
        '''Doc class, a representation of document.
        
        @param document: A Chinese sentence.
        @param tokenizer: the tokenizer(char/word)
        '''
        if tokenizer == 'char':
            self.bag_of_words = list(document)
        if tokenizer == 'word':
            self.bag_of_words = list(list(zip(*Doc.cut.cut(document)))[0])
        
#         self.vec = self.to_vec()
        Doc.corpus.append(self)
        #Doc.corpus.append(document)
    
    def to_vec(self):
        vec = np.zeros([512,512], dtype='float')
        for word in self.bag_of_words:
            if word in Doc.model.wv:
                vec += Doc.model.wv[word]
            else:
                for char in word:
                    vec += Doc.model.wv[char] if char in Doc.model.wv else 0

        return vec / np.linalg.norm(vec) if np.linalg.norm(vec) > 0 else vec

    def similarity(doc1, doc2):
        '''Return the cosine distance between two sentences.'''
        return np.dot(doc1.vec, doc2.vec)

    def most_similar(self):
        '''Find the most similar sentence in the corpus.

        Similar defined as cosine distance.
        '''
        most_simi, winner = 0, Doc('')

        for i, candidate in enumerate(Doc.corpus):
            simi = Doc.similarity(self, candidate)
            if simi > most_simi and candidate.bag_of_words != self.bag_of_words:
                most_simi, winner = simi, candidate

        return most_simi, ''.join(winner.bag_of_words)
    
    def test():
        '''Unit test & usage'''
        sentence1 = '天青色等烟雨'
        doc1 = Doc(sentence1)
        print('Tokenized and word vec[:10] of %s:' % sentence1)
        print(doc1.bag_of_words)
        print(doc1.to_vec()[:10])
        print('')
        print('Most similar word to 河流')
        print(Doc.model.most_similar('河流'))
        print('')
        sentence2 = '而我在等你'
        doc2 = Doc(sentence2)
        print('Similarity between %s, %s' % (sentence1, sentence2))
        print(Doc.similarity(doc1, doc2))
        print('')
        print('Most similar to %s in corpus' % sentence1)
        print(doc1.most_similar())
        print('')

Model loaded succeed


In [50]:
def load_lyrics(lyrics):
    count = 0
    Doc.corpus = []
    for lyric in lyrics:
        count += 1
        if count % 100 == 0:
            print (count)
#         for sentence in lyric:
        Doc(' '.join(lyric), tokenizer = 'word')
load_lyrics(test_lyrics)
bag = []
for doc in Doc.corpus:
    bag.append(' '.join(doc.bag_of_words))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000


In [22]:
Doc.test()

Tokenized and word vec[:10] of 天青色等烟雨:
['天', '青', '色', '等', '烟', '雨']
[[-0.0006566   0.00085584 -0.0003227  ..., -0.0006696  -0.002213
  -0.00104669]
 [-0.0006566   0.00085584 -0.0003227  ..., -0.0006696  -0.002213
  -0.00104669]
 [-0.0006566   0.00085584 -0.0003227  ..., -0.0006696  -0.002213
  -0.00104669]
 ..., 
 [-0.0006566   0.00085584 -0.0003227  ..., -0.0006696  -0.002213
  -0.00104669]
 [-0.0006566   0.00085584 -0.0003227  ..., -0.0006696  -0.002213
  -0.00104669]
 [-0.0006566   0.00085584 -0.0003227  ..., -0.0006696  -0.002213
  -0.00104669]]

Most similar word to 河流
[('奔腾', 0.675121545791626), ('稻香', 0.6594686508178711), ('原野', 0.6528366804122925), ('穿行', 0.6512413620948792), ('海洋', 0.6460176110267639), ('流淌啊', 0.6350507140159607), ('山间', 0.6339350938796997), ('旷野', 0.6277040839195251), ('江河', 0.6272122263908386), ('高原', 0.626851499080658)]

Similarity between 天青色等烟雨, 而我在等你


AttributeError: 'Doc' object has no attribute 'vec'

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD,NMF,LatentDirichletAllocation
from sklearn.cluster import KMeans

In [15]:
# original version
def find_topic(texts, topic_model, n_topics, vec_model="tf", thr=1e-2, **kwargs):
        # 1. vectorization
        vectorizer = CountVectorizer() if vec_model== "tf" else TfidfVectorizer()
        text_vec = vectorizer.fit_transform(texts)
        words = np.array(vectorizer.get_feature_names())

        # 2. topic finding
        topic_models = {"nmf":NMF, "svd": TruncatedSVD, "lda":LatentDirichletAllocation, "kmeans":KMeans}
        topicfinder = topic_models[topic_model](n_topics, **kwargs).fit(text_vec)
        topic_dists = topicfinder.components_ if topic_model is not "kmeans" else topicfinder.cluster_centers_
        topic_dists /= topic_dists.max(axis = 1).reshape((-1,1))
        
        # 3. keywords for topics
        def _topic_keywords(topic_dist):
            keywords_index = np.abs(topic_dist) >= thr
            keywords_prefix = np.where(np.sign(topic_dist)>0, "","^")[keywords_index]
            keywords = " | ".join(map(lambda x: "".join(x), zip(keywords_prefix, words[keywords_index])))
            return keywords
        topic_keywords = map(_topic_keywords, topic_dists)
        return "\n".join("Topic %i:%s" % (i, t) for i, t in enumerate(topic_keywords))

In [51]:
print(len(bag))

5000


In [54]:
# vec model is tf

#print(find_topic(bag,"svd",20, vec_model = "tf"))
# print(find_topic(bag,"nmf",20, vec_model = "tf"))
print(find_topic(bag,"lda",20, vec_model = "tf"))
# print(find_topic(bag,"kmeans",20, vec_model = "tf"))

# vec model is tfidf

#print(find_topic(bag,"svd",20, vec_model = "tfidf"))
# print(find_topic(bag, "nmf", 20, vec_model = "tfidf"))
#print(find_topic(bag,"lda",20, vec_model = "tfidf"))
# print(find_topic(bag,"kmeans",20, vec_model = "tfidf"))

/home/srthu2/anaconda3/envs/cqb/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


Topic 0:一丁点 | 一个 | 一二三 | 一十三 | 一年四季 | 一点点 | 一点点点 | 一痕 | 一直 | 一面之交 | 万古 | 三峡 | 三跪九叩 | 上山 | 上楼 | 下哗 | 下雨 | 不能自拔 | 不言不语 | 东乡 | 丝巾 | 两头 | 严肃 | 个人 | 个情人节 | 中华 | 中然自在 | 丰姿 | 临街 | 丹尼 | 丹尼别 | 为时 | 主人 | 主打 | 丽叶 | 久藏 | 乌龙把戏 | 乔晨 | 乞丐 | 乡味道 | 乱世九 | 云霞 | 五冬六夏 | 亦步亦趋 | 人情味 | 人民广场 | 介意 | 介绍 | 仔路 | 仙人 | 任子峰 | 任泪 | 伊万杜达尔 | 伊梨 | 优点 | 伟然 | 伤多 | 似水流年 | 佛恶 | 依据 | 依莎 | 侯佩岑 | 俏俏 | 俗尘 | 信会 | 信天游 | 信封 | 修个 | 倪安东 | 假正经 | 做事 | 做错 | 健华 | 偶像 | 偶像化 | 傍晚 | 先别 | 免疫 | 兔子 | 兜风 | 全球 | 其实 | 养身 | 冯雪刚 | 冰酒 | 冷子夕 | 冷字 | 冷峻 | 冷月 | 冷艳 | 冻知 | 凉山 | 凌波 | 几万 | 几百 | 凶勐 | 出手 | 出道 | 击中 | 击倒 | 分量 | 刘德欣 | 刘璇 | 到此为止 | 刹车 | 刺青 | 刻化 | 前夜 | 剩余 | 剪影 | 劝天 | 务事 | 劫祸 | 劲儿 | 千千片 | 千色 | 升华 | 单单 | 单单单单单 | 单身狗 | 卖杏 | 卡其色 | 卧木兰舟 | 即管 | 厕所 | 厝歹 | 县界 | 反反复 | 反弹 | 反覆无常 | 发挥 | 发白 | 发端 | 叙事 | 口袋 | 叫嚣 | 叭叭叭 | 叭嫌 | 可口 | 可敬 | 可爱 | 可能性 | 台南 | 台妹 | 台词 | 叶清 | 吆嚎 | 合伤 | 合别 | 合格 | 名人 | 名牌 | 吕洞宾 | 君手 | 听众 | 听心 | 吴亦帆 | 吴佳琳 | 吴依蔓 | 吹动 | 吻吻 | 周子琪 | 周筱婼 | 和尚 | 咕咕咕咕咕咕咕咕咕 | 咖啡豆 | 哎嘿 | 哎扎 | 哑巴 | 哥心窝 | 唇印 | 唉声叹气 | 唯思葵 | 唿召 | 唿咳 | 唿嗨 | 唿海 | 喊安 | 喋喋不休 | 喝口 | 嗓子 | 嗓门 | 嗯